In [2]:
from google.colab import drive
drive.mount('/content/drive')
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive
Mounted at /content/drive


In [1]:
!pip install face_recognition
!pip install opencv-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.1/100.1 MB 7.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566170 sha256=141f7a6f317c92790831b7c75fb9f1fbdc89eaedea5eabd233b1d8eb28d5f090
  Stored in directory: /root/.cache/pip/wheels/7a/eb/cf/e9eced74122b679557f597bb7c8e4c739cfcac526db1fd523d
Successfully built face-recognition-models


In [3]:
# import face_recognition as fr
import os
import cv2 # take input from the webcam, process it , give it to face ignition ..
#compare faces

import csv # handle csv file
import numpy as np

from datetime import datetime
#----------------------------- from debajyotikarmaker sir lecture
import tensorflow as tf
import matplotlib.pyplot as plt
import random
from tqdm import tqdm

In [4]:
Data_Path = '/content/drive/MyDrive/ImageData'
classes = os.listdir(Data_Path) # return list of folder  .. but for us we need to access a file

In [6]:
from tensorflow.keras.models import load_model

In [8]:
# Load the trained model
model = load_model('/content/updatedCodeFinal.h5')  # Replace 'your_model_path.h5' with the path to your trained model

model.summary()
#//////////////////////////////////////
#input_image = train_data[12][0]

# Resize the input image to match the model's input shape
#resized_image = cv2.resize(input_image, (128, 128))

# Expand dimensions to represent batch size (1 in this case)
#input_image_batch = np.expand_dims(resized_image, axis=0)

# Perform prediction using the model
#predictions = model.predict(input_image_batch)
#//////////////////////////////////////
#predictions = model.predict(train_data[12][0])
#predictions[0]
#np.argmax(predictions[0])

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 128, 128, 3)]        0         []                            
                                                                                                  
 Conv1 (Conv2D)              (None, 64, 64, 32)           864       ['input_2[0][0]']             
                                                                                                  
 bn_Conv1 (BatchNormalizati  (None, 64, 64, 32)           128       ['Conv1[0][0]']               
 on)                                                                                              
                                                                                                  
 Conv1_relu (ReLU)           (None, 64, 64, 32)           0         ['bn_Conv1[0][0]']        

In [9]:
# import dependencies
from IPython.display import display, Javascript, Image
from google.colab.output import eval_js
from base64 import b64decode, b64encode
import cv2
import numpy as np
import PIL
import io
import html
import time

# Helper Function
# function to convert the JavaScript object into an OpenCV image
def js_to_image(js_reply):
  """
  Params:
          js_reply: JavaScript object containing image from webcam
  Returns:
          img: OpenCV BGR image
  """
  # decode base64 image
  image_bytes = b64decode(js_reply.split(',')[1])
  # convert bytes to numpy array
  jpg_as_np = np.frombuffer(image_bytes, dtype=np.uint8)
  # decode numpy array into OpenCV BGR image
  img = cv2.imdecode(jpg_as_np, flags=1)

  return img

# function to convert OpenCV Rectangle bounding box image into base64 byte string to be overlayed on video stream
# def bbox_to_bytes(bbox_array):
#   """
#   Params:
#           bbox_array: Numpy array (pixels) containing rectangle to overlay on video stream.
#   Returns:
#         bytes: Base64 image byte string
#   """
#   # convert array into PIL image
#   bbox_PIL = PIL.Image.fromarray(bbox_array, 'RGBA')
#   iobuf = io.BytesIO()
#   # format bbox into png for return
#   bbox_PIL.save(iobuf, format='png')
#   # format return string
#   bbox_bytes = 'data:image/png;base64,{}'.format((str(b64encode(iobuf.getvalue()), 'utf-8')))

#   return bbox_bytes

# initialize the Haar Cascade face detection model
face_cascade = cv2.CascadeClassifier(cv2.samples.findFile(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'))

# JavaScript to properly create our live video stream using our webcam as input
def video_stream():
  js = Javascript('''
    var video;
    var div = null;
    var stream;
    var captureCanvas;
    var imgElement;
    var labelElement;

    var pendingResolve = null;
    var shutdown = false;

    function removeDom() {
       stream.getVideoTracks()[0].stop();
       video.remove();
       div.remove();
       video = null;
       div = null;
       stream = null;
       imgElement = null;
       captureCanvas = null;
       labelElement = null;
    }

    function onAnimationFrame() {
      if (!shutdown) {
        window.requestAnimationFrame(onAnimationFrame);
      }
      if (pendingResolve) {
        var result = "";
        if (!shutdown) {
          captureCanvas.getContext('2d').drawImage(video, 0, 0, 640, 480);
          result = captureCanvas.toDataURL('image/jpeg', 0.8)
        }
        var lp = pendingResolve;
        pendingResolve = null;
        lp(result);
      }
    }

    async function createDom() {
      if (div !== null) {
        return stream;
      }

      div = document.createElement('div');
      div.style.border = '2px solid black';
      div.style.padding = '3px';
      div.style.width = '100%';
      div.style.maxWidth = '600px';
      document.body.appendChild(div);

      const modelOut = document.createElement('div');
      modelOut.innerHTML = "<span>Status:</span>";
      labelElement = document.createElement('span');
      labelElement.innerText = 'No data';
      labelElement.style.fontWeight = 'bold';
      modelOut.appendChild(labelElement);
      div.appendChild(modelOut);

      video = document.createElement('video');
      video.style.display = 'block';
      video.width = div.clientWidth - 6;
      video.setAttribute('playsinline', '');
      video.onclick = () => { shutdown = true; };
      stream = await navigator.mediaDevices.getUserMedia(
          {video: { facingMode: "environment"}});
      div.appendChild(video);

      imgElement = document.createElement('img');
      imgElement.style.position = 'absolute';
      imgElement.style.zIndex = 1;
      imgElement.onclick = () => { shutdown = true; };
      div.appendChild(imgElement);

      const instruction = document.createElement('div');
      instruction.innerHTML =
          '<span style="color: red; font-weight: bold;">' +
          'When finished, click here or on the video to stop this demo</span>';
      div.appendChild(instruction);
      instruction.onclick = () => { shutdown = true; };

      video.srcObject = stream;
      await video.play();

      captureCanvas = document.createElement('canvas');
      captureCanvas.width = 640; //video.videoWidth;
      captureCanvas.height = 480; //video.videoHeight;
      window.requestAnimationFrame(onAnimationFrame);

      return stream;
    }
    async function stream_frame(label, imgData) {
      if (shutdown) {
        removeDom();
        shutdown = false;
        return '';
      }

      var preCreate = Date.now();
      stream = await createDom();

      var preShow = Date.now();
      if (label != "") {
        labelElement.innerHTML = label;
      }

      if (imgData != "") {
        var videoRect = video.getClientRects()[0];
        imgElement.style.top = videoRect.top + "px";
        imgElement.style.left = videoRect.left + "px";
        imgElement.style.width = videoRect.width + "px";
        imgElement.style.height = videoRect.height + "px";
        imgElement.src = imgData;
      }

      var preCapture = Date.now();
      var result = await new Promise(function(resolve, reject) {
        pendingResolve = resolve;
      });
      shutdown = false;

      return {'create': preShow - preCreate,
              'show': preCapture - preShow,
              'capture': Date.now() - preCapture,
              'img': result};
    }
    ''')

  display(js)

def video_frame(label, bbox):
  data = eval_js('stream_frame("{}", "{}")'.format(label, bbox))
  return data

In [10]:
classes = os.listdir(Data_Path)
print("class_names : 🎉🎉🎉", classes)

class_names : 🎉🎉🎉 ['AL-NAFI', 'ARNAB BISHAKH SARKER', 'ESM-E MOULA CHOWDHURY ABHA', 'ASHA ISLAM', 'ASHIK AHAMED', 'AHMED IMTIAZ', 'FAIZA BINTE ZAMAN', 'FAHIM RAHMAN', 'ABDULLAH AL SHAHRIAR', 'BISHANATH TARAFDER', 'MD ATIK ULLAH KHAN', 'MD NAFIZ AHMED', 'MD JAHID HASSAN', 'MD FARDIN AMIN RIYAD', 'MAHASHETAB HASSAN', 'IMAM HASAN JAMI', 'MD IMTIAJ ALAM SAJIN', 'MD MUNTASIR AREFIN NAEEM', 'MD ABU ZAYED KHAN', 'MD NAIMUR RAHMAN', 'MD SAJID ISLAM KHAN', 'MD SHANZID HASAN', 'MOHAMMAD BIN AB JALIL SHEAKH', 'MD NAZMUL HASSAN BHUIYAN', 'MD NAKIB MUNSIF', 'MD RIFAT SHIKDAR', 'MD WAHIDUZZAMAN SUVA', 'MD SAIFUR RAHMAN SADI', 'MOHAMMAD NUR', 'MD TOYABUR RAHAMAN', 'NAFIS MUBASSHIR SHAH', 'MOHAMMED TANVIR HASSAN', 'RAHAD-UL-ISLAM RABBY', 'REZWAN AHMAD', 'NAHAR ISLAM NISHI', 'SABBIR AHMED', 'S M FAISAL', 'NAVID MAHFUZ NAYEEM', 'MREDUL MAHMUD AKASH', 'MST ROKEYA KHATUN', 'ZOBAYER ALAM', 'SHEIKH AKIB ALMAS', 'TARIKUL ISLAM NISHAT', 'SATYAJIT DAS', 'SADAT BIN MASUD', 'TAHMID AL RAFID SIDDIQUE']


In [11]:
classes = os.listdir(Data_Path)
print("class_names : 🎉🎉🎉", classes)

# start streaming video from webcam
video_stream()
# label for video
label_html = 'Capturing...'
# initialze bounding box to empty
bbox = ''
count = 0
while True:
    js_reply = video_frame(label_html, bbox)
    if not js_reply:
        break

    # Convert JS response to OpenCV Image
    img = js_to_image(js_reply["img"])

    # Convert image to grayscale for face detection
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

    # Detect faces in the frame
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    # Process each detected face
    for (x, y, w, h) in faces:
        # Extract face region from the frame
        face_roi = gray[y:y+h, x:x+w]

        # Resize face image to match model input shape
        resized_face = cv2.resize(face_roi, (128, 128))

        # Normalize face image
        normalized_face = resized_face / 255.0

        # Expand dimensions to represent batch size (1 in this case)
        input_image_batch = np.expand_dims(normalized_face, axis=0)

        input_image_batch = np.repeat(input_image_batch, 3, axis=-1)  # Repeat the single-channel image to create three channels

        input_image_batch = input_image_batch.reshape(-1, 128, 128, 3) #-1,

        # Perform prediction using the model
        predictions = model.predict(input_image_batch)

        # Post-process predictions (if needed) and display results
        # For example, you can print the predicted class or display it on the frame
        predicted_class = np.argmax(predictions)
        predicted_class_name = classes[predicted_class]
        print("Predicted class : 🔰🔰🔰🔰🔰", classes[predicted_class] )
        #predicted_class_name =  class_names[predicted_class]
        #print("Predicted class: 🎉🎉🎉", predicted_class_name)
        # Draw bounding box around the face
        cv2.rectangle(img, (x, y), (x+w, y+h), (255, 0, 0), 2)


        # Display the predicted class name on the frame
        text = f'Predicted Class Name: {predicted_class_name}'
        cv2.putText(img, text, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)


        # Convert the frame with bounding boxes to bytes
        frame_bytes = cv2.imencode('.jpg', img)[1].tobytes()

class_names : 🎉🎉🎉 ['AL-NAFI', 'ARNAB BISHAKH SARKER', 'ESM-E MOULA CHOWDHURY ABHA', 'ASHA ISLAM', 'ASHIK AHAMED', 'AHMED IMTIAZ', 'FAIZA BINTE ZAMAN', 'FAHIM RAHMAN', 'ABDULLAH AL SHAHRIAR', 'BISHANATH TARAFDER', 'MD ATIK ULLAH KHAN', 'MD NAFIZ AHMED', 'MD JAHID HASSAN', 'MD FARDIN AMIN RIYAD', 'MAHASHETAB HASSAN', 'IMAM HASAN JAMI', 'MD IMTIAJ ALAM SAJIN', 'MD MUNTASIR AREFIN NAEEM', 'MD ABU ZAYED KHAN', 'MD NAIMUR RAHMAN', 'MD SAJID ISLAM KHAN', 'MD SHANZID HASAN', 'MOHAMMAD BIN AB JALIL SHEAKH', 'MD NAZMUL HASSAN BHUIYAN', 'MD NAKIB MUNSIF', 'MD RIFAT SHIKDAR', 'MD WAHIDUZZAMAN SUVA', 'MD SAIFUR RAHMAN SADI', 'MOHAMMAD NUR', 'MD TOYABUR RAHAMAN', 'NAFIS MUBASSHIR SHAH', 'MOHAMMED TANVIR HASSAN', 'RAHAD-UL-ISLAM RABBY', 'REZWAN AHMAD', 'NAHAR ISLAM NISHI', 'SABBIR AHMED', 'S M FAISAL', 'NAVID MAHFUZ NAYEEM', 'MREDUL MAHMUD AKASH', 'MST ROKEYA KHATUN', 'ZOBAYER ALAM', 'SHEIKH AKIB ALMAS', 'TARIKUL ISLAM NISHAT', 'SATYAJIT DAS', 'SADAT BIN MASUD', 'TAHMID AL RAFID SIDDIQUE']


<IPython.core.display.Javascript object>

1/1 [==============================] - 1s 1s/step
Predicted class : 🔰🔰🔰🔰🔰 IMAM HASAN JAMI
1/1 [==============================] - 0s 53ms/step
Predicted class : 🔰🔰🔰🔰🔰 ESM-E MOULA CHOWDHURY ABHA


KeyboardInterrupt: 